# Schools and vegan restaurants near companies

In [1]:
from pymongo import MongoClient
import pandas as pd
import functions as F
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [2]:
companies = list(db.cleaned_companies.find({}))
companies = pd.DataFrame(companies)

Filtering those companies which do not have airports or starbucks near

In [3]:
companies['airports'] = companies.location.apply(lambda x: F.geoQuery(db.airports, x))
companies['starbucks'] = companies.location.apply(lambda x: F.geoQuery(db.starbucks, x))

In [4]:
for col in ['airports','starbucks']:
    companies = companies[(companies[col].apply(F.nonempty))]

Create a school collection in mongodb and with google api search for schools close to the companies

In [5]:
db, school = F.connectMongo('school')
schools = []
schools_lista = F.near_place(companies, 1000, 'school',school, schools)

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.566879,-122.323895&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=40.757929,-73.985506&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=53.707739,10.023246&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.5645375,-122.3254705&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.772323,-122.214897&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=53.544711,-113.515769&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.412069,-121.900366&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.78424,-122.274771&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbys

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=42.3672873,-71.0814466&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=25.723408,-80.306331&radius=1000&keyword=school
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.369195,-122.036849&radius=1000&keyword=school


Create a vegan collection in mongodb and with google api search for vegan restaurants near the companies

In [6]:
db, vegan = F.connectMongo('vegan')
vegans = []
vegan_lista = F.near_place(companies, 1000, 'vegan',vegan, vegans)

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.566879,-122.323895&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=40.757929,-73.985506&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=53.707739,10.023246&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.5645375,-122.3254705&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.772323,-122.214897&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=53.544711,-113.515769&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.412069,-121.900366&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.78424,-122.274771&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/js

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=25.723408,-80.306331&radius=1000&keyword=vegan
https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.369195,-122.036849&radius=1000&keyword=vegan


Merging companies dataframe with vegan restaurants and school coordinates

In [7]:
vegan_df = pd.DataFrame(vegans)
school_df = pd.DataFrame(schools)

In [25]:
merged = companies.merge(vegan_df, left_on='_id', right_on='id')
merged = merged.merge(school_df, left_on='_id_x', right_on='id')
merged = merged.drop(columns = ['id_x','_id_y','id_y','_id'])

In [9]:
dist = merged[['airports','starbucks','location_x','location','location_y']].rename(columns ={'location_x':'location_companies','location':'location_schools','location_y':'location_vegan'})
dist = F.CleanDataFrame(dist,'airports').rename(columns={'location':'location_airports'})
dist = F.CleanDataFrame(dist,'starbucks').rename(columns={'location':'location_starbucks'})

Calculate distances using haversine function for each coordinate

In [10]:
dist = dist[['location_companies','location_schools','location_vegan','location_airports','location_starbucks']]
for i, row in dist.iterrows():
    for e in range(1,len(row)):
        distance = F.haversine(row[0]['coordinates'][1],row[0]['coordinates'][0],row[e]['coordinates'][1],row[e]['coordinates'][0])
        dist.loc[i,f'distance{e}'] = distance


In [23]:
dist.to_json('../OUTPUT/distances.json', orient = "records")
merged.to_json('../OUTPUT/merged.json', orient="records")

[-122.323895, 37.566879]